# TM10007 Assignment template

## Data loading and cleaning

Below are functions to load the dataset of your choice. After that, it is all up to you to create and evaluate a classification method. Beware, there may be missing values in these datasets. Good luck!

In [2]:
# # Run this to use from colab environment
# !pip install -q --upgrade git+https://github.com/karinvangarderen/tm10007_project.git

In [56]:
from load_data import load_data
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, chi2
import seaborn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from scipy import stats
from statsmodels.stats import weightstats

# Classifiers and kernels
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA, KernelPCA
from sklearn.kernel_approximation import RBFSampler
from sklearn.metrics.pairwise import rbf_kernel, sigmoid_kernel
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB

data = load_data()
print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')

features = data.drop(columns=['label'])
label = data.label

# Splitting data in train and test group
X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=.2)

# functie van maken??
y_train_bin = []
for val in y_train:
  if val == 'T12':
    y_train_bin.append(0)
  else:
    y_train_bin.append(1) 


The number of samples: 113
The number of columns: 160


## Preprocessing

### Scaling

In [57]:
# Scale the dataset
scaler = RobustScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Feature selection

### T-test

In [58]:
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns = X_train.columns) # make df from numpy
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns = X_train.columns)
X_train_scaled_df['Label'] = y_train_bin
X_train_T12 = X_train_scaled_df.groupby('Label').get_group(0)
X_train_T34 = X_train_scaled_df.groupby('Label').get_group(1)
X_train_T12 = X_train_T12.drop(columns = ['Label'])
X_train_T34 = X_train_T34.drop(columns = ['Label'])

# ttest
_,pval = stats.ttest_ind(X_train_T12,X_train_T34)



sig_feat = []
for id, val in enumerate(pval):
  if val < 0.05/X_train_scaled_df.shape[1]:
    sig_feat.append(list(X_train.columns)[id])
print(f'Number of significant different features: {len(sig_feat)}')

X_train_sig = X_train_scaled_df[sig_feat]
X_test_sig = X_test_scaled_df[sig_feat]

# # Pairplot of sign features
# X_train_sig.columns =['Feature'+ str(pc) for pc in range(1,len(sig_feat)+1)]
# X_train_sig['Grade'] = y_train_bin
# pair_plot = seaborn.pairplot(X_train_sig, hue = 'Grade')

Number of significant different features: 25


## PCA

In [59]:
N_COMP = 10
pca = PCA(n_components=N_COMP)
pca.fit(X_train_sig)
X_train_pca = pca.transform(X_train_sig)
X_test_pca = pca.transform(X_test_sig)


# seaborn.scatterplot(x=X_train_pca[:,0],y=X_train_pca[:,1],hue=y_train)
# scatter_data = pd.DataFrame(X_train_pca[:,:], columns = ['Principal component' + str(pc) for pc in range(1,N_COMP+1)])
# scatter_data['Stage'] = y_train_bin
# seaborn.pairplot(scatter_data, hue = 'Stage')
# print(scatter_data)

# Classifier

In [60]:
# Construct classifiers
svmlin = SVC(kernel='linear', gamma='scale')
svmrbf = SVC(kernel='rbf', gamma='scale')
svmpoly = SVC(kernel='poly', degree=3, gamma='scale')

clsfs = [KNeighborsClassifier(), RandomForestClassifier(),QuadraticDiscriminantAnalysis(),GaussianNB(),LinearDiscriminantAnalysis(),svmlin, svmpoly, svmrbf]

for clf in clsfs:
    # Fit classifier
    clf.fit(X_train_sig,y_train)
    y_pred_train=clf.predict(X_train_sig)
    print(clf)
    acc_train = (y_train==y_pred_train).sum()/len(X_train_sig)
    print(f'Train data acc: {acc_train}')
    y_pred_test = clf.predict(X_test_sig)
    acc_test = (y_test==y_pred_test).sum()/len(X_test_sig)
    print(f'Test data acc: {acc_test}')

# clsfs = [KNeighborsClassifier(n_neighbors=5, weights='distance'),KNeighborsClassifier(n_neighbors=5),KNeighborsClassifier(n_neighbors=5),KNeighborsClassifier(n_neighbors=10)]

# for clf in clsfs:
#     # Fit classifier
#     clf.fit(X_train_sig,y_train)
#     y_pred_train=clf.predict(X_train_sig)
#     print(clf)
#     acc_train = (y_train==y_pred_train).sum()/len(X_train_sig)
#     print(f'Train data acc: {acc_train}')
#     y_pred_test = clf.predict(X_test_sig)
#     acc_test = (y_test==y_pred_test).sum()/len(X_test_sig)
#     print(f'Test data acc: {acc_test}')


KNeighborsClassifier()
Train data acc: 0.7888888888888889
Test data acc: 0.7391304347826086
RandomForestClassifier()
Train data acc: 1.0
Test data acc: 0.6521739130434783
QuadraticDiscriminantAnalysis()
Train data acc: 0.9222222222222223
Test data acc: 0.6086956521739131
GaussianNB()
Train data acc: 0.7555555555555555
Test data acc: 0.6521739130434783
LinearDiscriminantAnalysis()
Train data acc: 0.8555555555555555
Test data acc: 0.5217391304347826
SVC(kernel='linear')
Train data acc: 0.8222222222222222
Test data acc: 0.6956521739130435
SVC(kernel='poly')
Train data acc: 0.7444444444444445
Test data acc: 0.6956521739130435
SVC()
Train data acc: 0.8222222222222222
Test data acc: 0.6956521739130435


In [61]:
-

SyntaxError: invalid syntax (476313318.py, line 1)